In [1]:
import pandas as pd
import glob
import os
import numpy as np

In [2]:
# VARIABLES DE ENTORNO
# ADVERTENCIA: La calificación de corte con la que están los archivos base es de 80. Por lo que si se modifica
# quizá haya más personas en ciclos más avanzados, pero el promedio de notas para esos examenes seguirá igual
# debido a que esas personas adicionales no presentaron dicha prueba.
CALIFICACION_CORTE = 80
KEY_DFS = 'datos'
KEY_NOMBRE_CURSO = 'nombre curso'
KEY_TOTAL_EST = 'total tripulantes'
KEY_PROMEDIOS_CALIFICACIONES = 'promedios'
KEY_ACOGIDA = 'acogida'
# Es importante que las columnas de las calificaciones vayan en el orden ascendente lógico de la prueba.
COLUMNAS_CALIFICACIONES = [
    'Cuestionario:Examen Ciclo 1 (Real)',
    'Cuestionario:Examen Ciclo 2 (Real)',
    'Cuestionario:Examen Ciclo 3 (Real)'
]
COLUMNA_CICLO_INICIAL = 'Ciclo en que inicia'

In [3]:
# LECTURA DE DATOS
archivos = glob.glob('./INPUT/*.xlsx')
cursos = {}
for archivo in archivos:
    nombreArchivo = os.path.basename(archivo)
    nombreCurso = nombreArchivo.split('-')[3]
    cursos[nombreCurso] = {KEY_DFS: pd.read_excel(archivo, engine = 'openpyxl')}
    for columna in COLUMNAS_CALIFICACIONES:
        cursos[nombreCurso][KEY_DFS][columna] = pd.to_numeric(cursos[nombreCurso][KEY_DFS][columna], errors='coerce')

In [4]:
# PROCESADO DE DATOS
for curso in cursos:
    # Total estudiantes
    # Total acogida (estudiantes que sí presentaron)
    # Promedio ciclos
    # Ciclo en el que queda cada estudiante
    cursos[curso][KEY_NOMBRE_CURSO] = curso
    cursos[curso][KEY_TOTAL_EST] = len(cursos[curso][KEY_DFS])
    cursos[curso][KEY_ACOGIDA] = len(cursos[curso][KEY_DFS][cursos[curso][KEY_DFS][COLUMNAS_CALIFICACIONES[0]].isnull() == True])
    cursos[curso][KEY_PROMEDIOS_CALIFICACIONES] = []
    for columna in COLUMNAS_CALIFICACIONES:
        cursos[curso][KEY_PROMEDIOS_CALIFICACIONES].append(cursos[curso][KEY_DFS][columna].mean())
    mayorCiclo = []
    for index, row in cursos[curso][KEY_DFS].iterrows():
        ciclo = 1
        for columna in COLUMNAS_CALIFICACIONES:
            if row[columna] >= CALIFICACION_CORTE:
                ciclo += 1
            else:
                break
        mayorCiclo.append('C' + str(ciclo))
    cursos[curso][KEY_DFS][COLUMNA_CICLO_INICIAL] = pd.Series(mayorCiclo)

In [ ]:
# RESULTADOS POR CURSO
dfs = []
totalCiclos = {}
for curso in cursos:
    print ('================================')
    print ('Curso:', cursos[curso][KEY_NOMBRE_CURSO])
    print ('Total de tripulantes que NO presentaron la prueba:', cursos[curso][KEY_ACOGIDA])
    print ('Total de tripulantes que presentaron la prueba:', cursos[curso][KEY_TOTAL_EST] - cursos[curso][KEY_ACOGIDA])
    print ('Total de tripulantes:', cursos[curso][KEY_TOTAL_EST])
    print ('\n')
    for i in range(1,4):
        ciclo = 'C' + str(i)
        print ('Promedio de calificaciones en', ciclo, ':', cursos[curso][KEY_PROMEDIOS_CALIFICACIONES][i-1])
        num = len(cursos[curso][KEY_DFS][cursos[curso][KEY_DFS][COLUMNA_CICLO_INICIAL] == ciclo])
        if ciclo in totalCiclos:
            totalCiclos[ciclo] += num
        else:            
            totalCiclos[ciclo] = num
        print ('Estudiantes que quedaron en', ciclo, ':', num)
        print ('------')
    cursos[curso][KEY_DFS].to_excel('./OUTPUT/resultados_' + curso + '_corte_'+ str(CALIFICACION_CORTE) +'.xlsx', index = False)
    dfs.append(cursos[curso][KEY_DFS])
pd.concat(dfs, ignore_index = True).to_excel("./OUTPUT/CONSOLIDADO.xlsx", index = False)
for ciclo in totalCiclos:
    print ('Estudiantes que quedaron en el ciclo',ciclo, ':', totalCiclos[ciclo])